<a href="https://colab.research.google.com/github/kridtapon/Smart-Money-Sweep-Strategy-SMSS-/blob/main/Smart_Money_Sweep_Strategy_(SMSS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Cumulative Volume Delta (CVD)
def calculate_cvd(df, shift=1):
    """
    Cumulative Volume Delta (CVD) is calculated using the difference between up and down volume.
    """
    df['Up_Vol'] = np.where(df['Close'] > df['Close'].shift(shift), df['Volume'], 0)
    df['Down_Vol'] = np.where(df['Close'] < df['Close'].shift(shift), df['Volume'], 0)
    df['CVD'] = (df['Up_Vol'] - df['Down_Vol']).cumsum()
    return df

# Function to detect CVD Signal
def detect_cvd_signal(df, shift_bearish=1, shift_bullish=2):
    """
    Detects CVD Signal where price and CVD move in opposite directions.
    """
    df['Bearish_CVD_Signal'] = (df['High'] > df['High'].shift(shift_bearish)) & (df['CVD'] < df['CVD'].shift(shift_bearish))
    df['Bullish_CVD_Signal'] = (df['Low'] < df['Low'].shift(shift_bullish)) & (df['CVD'] > df['CVD'].shift(shift_bullish))
    return df

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """
    Calculates the MACD indicator and its signal line.
    """
    df['EMA_Short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['EMA_Long'] = df['Close'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = df['EMA_Short'] - df['EMA_Long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    return df

# Download historical data
symbol = 'ANET'
start_date = '2019-01-01'
end_date = '2025-01-01'

df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate CVD
df = calculate_cvd(df, shift=1)

# Detect CVD divergence
df = detect_cvd_signal(df, shift_bearish=1, shift_bullish=2)

# Calculate MACD
df = calculate_macd(df, short_window=12, long_window=26, signal_window=9)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Define Entry and Exit Conditions using CVD and MACD
df['MACD_Cross_Above'] = (df['MACD'] > df['MACD_Signal'])
df['MACD_Cross_Below'] = (df['MACD'] < df['MACD_Signal'])

df['Entry'] = df['Bullish_CVD_Signal'] & df['MACD_Cross_Above']
df['Exit'] = df['Bearish_CVD_Signal'] & df['MACD_Cross_Below']

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           325314.393053
Total Return [%]                       225.314393
Benchmark Return [%]                   763.853053
Max Gross Exposure [%]                      100.0
Total Fees Paid                       7629.231076
Max Drawdown [%]                        21.413738
Max Drawdown Duration                       283.0
Total Trades                                   21
Total Closed Trades                            20
Total Open Trades                               1
Open Trade PnL                       29011.122136
Win Rate [%]                                 60.0
Best Trade [%]                          43.986209
Worst Trade [%]                        -13.965106
Avg Winning Trade [%]                   14.364549
Avg Losing Trade [%]                     -5.34395


In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Cumulative Volume Delta (CVD)
def calculate_cvd(df, shift=1):
    df['Up_Vol'] = np.where(df['Close'] > df['Close'].shift(shift), df['Volume'], 0)
    df['Down_Vol'] = np.where(df['Close'] < df['Close'].shift(shift), df['Volume'], 0)
    df['CVD'] = (df['Up_Vol'] - df['Down_Vol']).cumsum()
    return df

# Function to detect CVD Signal
def detect_cvd_signal(df, shift_bearish=1, shift_bullish=2):
    df['Bearish_CVD_Signal'] = (df['High'] > df['High'].shift(shift_bearish)) & (df['CVD'] < df['CVD'].shift(shift_bearish))
    df['Bullish_CVD_Signal'] = (df['Low'] < df['Low'].shift(shift_bullish)) & (df['CVD'] > df['CVD'].shift(shift_bullish))
    return df

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    df['EMA_Short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['EMA_Long'] = df['Close'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = df['EMA_Short'] - df['EMA_Long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    return df

# Download historical data
symbol = 'ANET'
start_date = '2019-01-01'
end_date = '2025-01-01'

df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Define the parameter grid for optimization
param_grid = {
    'shift': range(1, 10, 2),
    'shift_bearish': range(1, 10, 2),
    'shift_bullish': range(1, 10, 2),
    'short_window': range(5, 20, 2),
    'long_window': range(20, 31, 2),
    'signal_window': range(5, 20, 2)
}

# Create a function to calculate portfolio performance based on the given parameters
def optimize_strategy(params):
    # Extract parameters
    shift = params['shift']
    shift_bearish = params['shift_bearish']
    shift_bullish = params['shift_bullish']
    short_window = params['short_window']
    long_window = params['long_window']
    signal_window = params['signal_window']

    # Calculate CVD and MACD with given parameters
    df_opt = df.copy()
    df_opt = calculate_cvd(df_opt, shift=shift)
    df_opt = detect_cvd_signal(df_opt, shift_bearish=shift_bearish, shift_bullish=shift_bullish)
    df_opt = calculate_macd(df_opt, short_window=short_window, long_window=long_window, signal_window=signal_window)

    # Define Entry and Exit Conditions
    df_opt['MACD_Cross_Above'] = (df_opt['MACD'] > df_opt['MACD_Signal'])
    df_opt['MACD_Cross_Below'] = (df_opt['MACD'] < df_opt['MACD_Signal'])

    # Filter data for the test period (2020-2025)
    df_filtered = df_opt[(df_opt.index.year >= 2020) & (df_opt.index.year <= 2025)]

    df_filtered['Entry'] = df_filtered['Bullish_CVD_Signal'] & df_filtered['MACD_Cross_Above']
    df_filtered['Exit'] = df_filtered['Bearish_CVD_Signal'] & df_filtered['MACD_Cross_Below']

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_filtered['Close'],
        entries=df_filtered['Entry'],
        exits=df_filtered['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    # Return performance stats (e.g., total returns)
    return portfolio.stats()['Total Return [%]']

# Use itertools to generate all combinations of the parameter grid
param_combinations = itertools.product(
    param_grid['shift'],
    param_grid['shift_bearish'],
    param_grid['shift_bullish'],
    param_grid['short_window'],
    param_grid['long_window'],
    param_grid['signal_window']
)

best_params = None
best_return = -np.inf

# Loop over parameter combinations and find the best parameters
for combination in param_combinations:
    params = {
        'shift': combination[0],
        'shift_bearish': combination[1],
        'shift_bullish': combination[2],
        'short_window': combination[3],
        'long_window': combination[4],
        'signal_window': combination[5]
    }

    return_ = optimize_strategy(params)

    # Update the best parameters if the current one is better
    if return_ > best_return:
        best_return = return_
        best_params = params

# Output the best parameters and performance
print(f"Best Parameters: {best_params}")
print(f"Best Total Return: {best_return}")


Streaming output truncated to the last 5000 lines.
<ipython-input-9-d7afec529163>:70: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

<ipython-input-9-d7afec529163>:69: SettingWi

Best Parameters: {'shift': 9, 'shift_bearish': 5, 'shift_bullish': 3, 'short_window': 5, 'long_window': 20, 'signal_window': 7}
Best Total Return: 1149.1767583680846


/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

<ipython-input-9-d7afec529163>:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-d7afec529163>:70: SettingWithCopyWarning:


A value is trying to be set on a c

In [10]:
# Calculate CVD
df = calculate_cvd(df, shift=9)

# Detect CVD divergence
df = detect_cvd_signal(df, shift_bearish=5, shift_bullish=3)

# Calculate MACD
df = calculate_macd(df, short_window=5, long_window=20, signal_window=7)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Define Entry and Exit Conditions using CVD and MACD
df['MACD_Cross_Above'] = (df['MACD'] > df['MACD_Signal'])
df['MACD_Cross_Below'] = (df['MACD'] < df['MACD_Signal'])

df['Entry'] = df['Bullish_CVD_Signal'] & df['MACD_Cross_Above']
df['Exit'] = df['Bearish_CVD_Signal'] & df['MACD_Cross_Below']

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

<ipython-input-10-ff3c7b7acf36>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-ff3c7b7acf36>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-ff3c7b7acf36>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-ff3c

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                          1249176.758368
Total Return [%]                      1149.176758
Benchmark Return [%]                   763.853053
Max Gross Exposure [%]                      100.0
Total Fees Paid                      12793.038499
Max Drawdown [%]                        25.833264
Max Drawdown Duration                       147.0
Total Trades                                   18
Total Closed Trades                            17
Total Open Trades                               1
Open Trade PnL                      111399.988074
Win Rate [%]                            88.235294
Best Trade [%]                          68.177126
Worst Trade [%]                         -7.722824
Avg Winning Trade [%]                     19.5218
Avg Losing Trade [%]                    -5.000066
